In [3]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [31]:
import numpy as np

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [32]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model

import pandas as pd

list_of_train = ['number_of_slot','number_of_day','number_of_weekday','number_of_hour']
list_of_target = ['stream1','stream2','stream3','stream4','stream5','bandwidth_available_OTT']

data = pd.read_csv("train_data.csv")

#lst_x = []
#for i in range(len(data)):
 #   lst_x.append([data.number_of_slot[i], data.number_of_day[i], data.number_of_weekday[i], data.number_of_hour[i]])

#lst_y = []
#for i in range(len(data)):
 #   lst_y.append([round(data.stream1[i],6) , round(data.stream2[i], 6), round(data.stream3[i], 6),  round(data.stream4[i], 6), round(data.stream5[i], 6), round(data.bandwidth_available_OTT[i], 6)])

x = data[list_of_train]

y = data[list_of_target]

rf_random.fit(x, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 14.8min
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

In [ ]:
rf_random.best_params_

In [39]:
import pandas as pd
import numpy as np

list_of_train = ['number_of_slot','number_of_day','number_of_weekday','number_of_hour']
list_of_target = ['stream1','stream2','stream3','stream4','stream5','bandwidth_available_OTT']

data = pd.read_csv("train_data.csv")

#lst_x = []
#for i in range(len(data)):
 #   lst_x.append([data.number_of_slot[i], data.number_of_day[i], data.number_of_weekday[i], data.number_of_hour[i]])

#lst_y = []
#for i in range(len(data)):
 #   lst_y.append([round(data.stream1[i],6) , round(data.stream2[i], 6), round(data.stream3[i], 6),  round(data.stream4[i], 6), round(data.stream5[i], 6), round(data.bandwidth_available_OTT[i], 6)])

x = data[list_of_train]

y = data[list_of_target]

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

#clf = RandomForestClassifier()

reg = RandomForestRegressor(n_estimators=400, min_samples_split=10, min_samples_leaf=4,max_features='auto',max_depth=70,bootstrap=True)

x_d = pd.read_csv("test_data.csv")

#lst_test_x = []
#for i in range(len(x_d)):
 #   lst_test_x.append([x_d.number_of_slot[i], x_d.number_of_day[i], x_d.number_of_weekday[i], x_d.number_of_hour[i]])

test_x = x_d[list_of_train]

# fit the model to the training data (learn the coefficients)
reg.fit(x, y)

# make predictions on the testing set
y_pred = reg.predict(test_x)

lst = []
for i in range(len(y_pred)):
    round_lst = []
    for j in range(len(y_pred[i])):
        round_lst.append(round(y_pred[i][j], 6))
        
    lst.append(round_lst)

predstream = np.zeros((2256,5))

for i in range(2256):
    for j in range(5):
        predstream[i,j] =y_pred[i,j] 
        
m = np.zeros((2256,1))
max_val = np.zeros((2256,1))
for k in range(2256):
    for l in range(5):
        if max_val[k] < predstream[k,l]:
            max_val[k] =  predstream[k,l]
            m[k] = l

for i in range(len(m)):
    print(m[i])



[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[3.]
[3.]
[1.]
[1.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[0.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[1.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]


[2.]
[2.]
[2.]
[2.]
[2.]
[3.]
[3.]
[1.]
[1.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[0.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[1.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[3.]
[3.]
[1.]
[1.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]


In [28]:
##### import pandas as pd
import numpy as np

list_of_train = ['number_of_slot','number_of_day','number_of_weekday','number_of_hour']
list_of_target = ['stream1','stream2','stream3','stream4','stream5','bandwidth_available_OTT']

data = pd.read_csv("train_data.csv")

x = data[list_of_train]

y = data[list_of_target]

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    print(predictions)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format((np.mean(errors))))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    #print('Accuracy = ' , accuracy)
    
    return accuracy

base_model = RandomForestRegressor(n_estimators=800, min_samples_split=2, min_samples_leaf=2,max_features='auto',max_depth=20,bootstrap=True)
base_model.fit(x, y)

data = pd.read_csv("testing_data.csv")

lst_x = []
for i in range(len(data)):
    lst_x.append([data.number_of_slot[i], data.number_of_day[i], data.number_of_weekday[i], data.number_of_hour[i]])

lst_y = []
for i in range(len(data)):
    lst_y.append([round(data.stream1[i],6) , round(data.stream2[i], 6), round(data.stream3[i], 6),  round(data.stream4[i], 6), round(data.stream5[i], 6), round(data.bandwidth_available_OTT[i], 6)])

base_accuracy = evaluate(base_model, lst_x, lst_y)

[[0.00000000e+00 1.76769564e-01 1.14814588e+00 3.87733307e-01
  0.00000000e+00 4.98294216e+00]
 [0.00000000e+00 3.16479940e-01 8.19543566e-01 4.66043900e-01
  0.00000000e+00 6.21371036e+00]
 [0.00000000e+00 6.02735511e-01 1.12710124e+00 1.75566652e-01
  0.00000000e+00 6.61017957e+00]
 ...
 [7.87984078e-01 2.38840705e-04 1.23137561e+00 9.95320540e-01
  2.54733668e+00 9.13436199e-01]
 [5.09633682e-01 2.02889015e-03 1.10308367e+00 5.76827632e-01
  4.62582383e+00 1.39666983e+00]
 [7.16319673e-01 1.37479048e-02 4.89850804e-01 6.86298488e-01
  2.83013888e+00 2.95327994e+00]]
Model Performance
Average Error: 0.1765 degrees.


C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


In [102]:
import numpy as np

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [100, 200, 300, 1000]
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(80, 110, num = 4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [8, 10, 12]
# Minimum number of samples required at each leaf node
min_samples_leaf = [3, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True],
 'max_depth': [80, 90, 100, 110, None],
 'max_features': ['auto'],
 'min_samples_leaf': [3, 4, 5],
 'min_samples_split': [8, 10, 12],
 'n_estimators': [100, 200, 300, 1000]}


In [114]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model

import pandas as pd

list_of_train = ['number_of_slot','number_of_day','number_of_weekday','number_of_hour']
#list_of_target = ['stream1','stream2','stream3','stream4','stream5','bandwidth_available_OTT']
list_of_target = ['stream2']

data = pd.read_csv("train_data.csv")

#lst_x = []
#for i in range(len(data)):
 #   lst_x.append([data.number_of_slot[i], data.number_of_day[i], data.number_of_weekday[i], data.number_of_hour[i]])

#lst_y = []
#for i in range(len(data)):
 #   lst_y.append([round(data.stream1[i],6) , round(data.stream2[i], 6), round(data.stream3[i], 6),  round(data.stream4[i], 6), round(data.stream5[i], 6), round(data.bandwidth_available_OTT[i], 6)])

x = data[list_of_train]

y = data[list_of_target]

rf_random.fit(x, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 17.0min finished
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [116]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 10,
 'bootstrap': True}

In [107]:
import pandas as pd
import numpy as np

list_of_train = ['number_of_slot','number_of_day','number_of_weekday','number_of_hour']
#list_of_target = ['stream1','stream2','stream3','stream4','stream5','bandwidth_available_OTT']
list_of_target = ['stream1']

data = pd.read_csv("testing_data.csv")

x = data[list_of_train]

y = data[list_of_target]

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format((np.mean(errors)))
    print('Average Error: ' , np.mean(errors))
    print('Accuracy = ' , accuracy)
    
    return accuracy

base_model = RandomForestRegressor(n_estimators=200, min_samples_split=12, min_samples_leaf=5,max_features='auto',max_depth=110,bootstrap=True)
base_model.fit(x, y)
base_accuracy = evaluate(base_model, x, y)

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


ValueError: Unable to coerce to Series, length must be 1: given 2256

In [96]:

base_model = RandomForestRegressor(n_estimators=400, min_samples_split=6, min_samples_leaf=4,max_features='auto',max_depth=40,bootstrap=True)
base_model.fit(x, y)
base_accuracy = evaluate(base_model, x, y)

Model Performance
Average Error:  stream1                    0.091030
stream2                    0.099236
stream3                    0.469849
stream4                    0.131067
stream5                    0.374881
bandwidth_available_OTT    0.130485
dtype: float64
Accuracy =  stream1                         -inf
stream2                         -inf
stream3                    35.182494
stream4                         -inf
stream5                    83.843340
bandwidth_available_OTT         -inf
dtype: float64


In [30]:
import pandas as pd
import numpy as np

list_of_train = ['number_of_slot','number_of_day','number_of_weekday','number_of_hour']
list_of_target = ['stream1','stream2','stream3','stream4','stream5','bandwidth_available_OTT']
#list_of_target = ['bandwidth_available_OTT']

data = pd.read_csv("train_data.csv")

#lst_x = []
#for i in range(len(data)):
 #   lst_x.append([data.number_of_slot[i], data.number_of_day[i], data.number_of_weekday[i], data.number_of_hour[i]])

#lst_y = []
#for i in range(len(data)):
 #   lst_y.append([round(data.stream1[i],6) , round(data.stream2[i], 6), round(data.stream3[i], 6),  round(data.stream4[i], 6), round(data.stream5[i], 6), round(data.bandwidth_available_OTT[i], 6)])

x = data[list_of_train]

y = data[list_of_target]

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

#clf = RandomForestClassifier()

reg = RandomForestRegressor(n_estimators=800, min_samples_split=2, min_samples_leaf=2,max_features='auto',max_depth=20,bootstrap=True)

x_d = pd.read_csv("test_data.csv")

#lst_test_x = []
#for i in range(len(x_d)):
 #   lst_test_x.append([x_d.number_of_slot[i], x_d.number_of_day[i], x_d.number_of_weekday[i], x_d.number_of_hour[i]])

test_x = x_d[list_of_train]

# fit the model to the training data (learn the coefficients)
reg.fit(x, y)

# make predictions on the testing set
y_pred = reg.predict(test_x)

lst = []
for i in range(len(y_pred)):
    round_lst = []
    for j in range(len(y_pred[i])):
        round_lst.append(round(y_pred[i][j], 6))
        
    lst.append(round_lst)

predstream = np.zeros((2256,5))

for i in range(2256):
    for j in range(5):
        predstream[i,j] =y_pred[i,j] 
        
m = np.zeros((2256,1))
max_val = np.zeros((2256,1))
for k in range(2256):
    for l in range(5):
        if max_val[k] < predstream[k,l]:
            max_val[k] =  predstream[k,l]
            m[k] = l

for i in range(len(m)):
    print(m[i])

'''
for i in range(len(lst)):
    print(lst[i])
'''

[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[0.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[3.]
[3.]
[1.]
[1.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[0.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[1.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[0.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]


[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[2.]
[2.]
[2.]
[3.]
[3.]
[1.]
[1.]
[3.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[2.]
[2.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[1.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[0.]
[2.]
[2.]
[4.]
[0.]
[4.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[1.]
[4.]
[1.]
[1.]
[1.]
[1.]
[1.]
[2.]
[1.]
[1.]
[2.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[0.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[2.]
[2.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[4.]
[1.]
[4.]
[4.]
[4.]


'\nfor i in range(len(lst)):\n    print(lst[i])\n'

In [58]:
import pandas as pd
import numpy as np

data = pd.read_csv("submission_pikapika.csv")

lst = []
for i in range(len(data)):
    lst.append([round(data.stream1[i],6) , round(data.stream2[i], 6), round(data.stream3[i], 6),  round(data.stream4[i], 6), round(data.stream5[i], 6)])

#5-1-3-4-2

#for
new_lst = []
for i in range(len(lst)):
    mark = 0
    max_value = lst[i].index(max(lst[i]))
    for k in range(len(lst[i])):
            if(abs(max_value - lst[i][k]) <= 0.99):
                if((lst[i].index(lst[i][j]) == 0 and lst[i].index(lst[i][k]) == 1) or (lst[i].index(lst[i][k]) == 0 and lst[i].index(lst[i][j]) == 1)):
                    mark = 1
                    new_lst.append(1)
                elif((lst[i].index(lst[i][j]) == 0 and lst[i].index(lst[i][k]) == 2) or (lst[i].index(lst[i][k]) == 0 and lst[i].index(lst[i][j]) == 2)):
                    mark = 1
                    new_lst.append(1)
                elif((lst[i].index(lst[i][j]) == 0 and lst[i].index(lst[i][k]) == 3) or (lst[i].index(lst[i][k]) == 0 and lst[i].index(lst[i][j]) == 3)):
                    mark = 1
                    new_lst.append(1)
                elif((lst[i].index(lst[i][j]) == 0 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 0 and lst[i].index(lst[i][j]) == 4)):
                    mark = 1
                    new_lst.append(5)
                elif((lst[i].index(lst[i][j]) == 1 and lst[i].index(lst[i][k]) == 2) or (lst[i].index(lst[i][k]) == 1 and lst[i].index(lst[i][j]) == 2)):
                    mark = 1
                    new_lst.append(3)
                elif((lst[i].index(lst[i][j]) == 1 and lst[i].index(lst[i][k]) == 3) or (lst[i].index(lst[i][k]) == 1 and lst[i].index(lst[i][j]) == 3)):
                    mark = 1
                    new_lst.append(4)
                elif((lst[i].index(lst[i][j]) == 1 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 1 and lst[i].index(lst[i][j]) == 4)):
                    mark = 1
                    new_lst.append(5)
                elif((lst[i].index(lst[i][j]) == 2 and lst[i].index(lst[i][k]) == 3) or (lst[i].index(lst[i][k]) == 2 and lst[i].index(lst[i][j]) == 3)):
                    mark = 1
                    new_lst.append(3)
                elif((lst[i].index(lst[i][j]) == 2 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 2 and lst[i].index(lst[i][j]) == 4)):
                    mark = 1
                    new_lst.append(5)
                elif((lst[i].index(lst[i][j]) == 3 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 3 and lst[i].index(lst[i][j]) == 4)):
                    mark = 1
                    new_lst.append(5)
                    
    if(mark == 0):
        max_value = max_value + 1
        new_lst.append(max_value)
                
for i in range(len(new_lst)):
    print(new_lst[i])



5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
3
3
1
1
1
4
4
1
1
1
1
1
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
3
3
1
1
1
4
4
1
1
1
1
1
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


In [88]:
import pandas as pd
import numpy as np

data = pd.read_csv("submission_pikapika.csv")

lst = []
for i in range(len(data)):
    lst.append([data.stream1[i] , data.stream2[i], data.stream3[i],  data.stream4[i], data.stream5[i]])

print(len(lst))
#5-1-3-4-2

#for
new_lst = []
mark1_lst = []
mark_lst = []
for i in range(len(lst)):
    mark = 0
    max_value = lst[i].index(max(lst[i]))

    for k in range(len(lst[i])):
        if((abs(max(lst[i]) - lst[i][k]) <= 0.9) and (max(lst[i]) != lst[i][k])):
                if((max_value == 0 and lst[i].index(lst[i][k]) == 1) or (lst[i].index(lst[i][k]) == 0 and max_value == 1)):
                    mark = 1
                    new_lst.append(1)
                    break
                elif((max_value == 0 and lst[i].index(lst[i][k]) == 2) or (lst[i].index(lst[i][k]) == 0 and max_value == 2)):
                    mark = 1
                    new_lst.append(1)
                    break
                elif((max_value == 0 and lst[i].index(lst[i][k]) == 3) or (lst[i].index(lst[i][k]) == 0 and max_value == 3)):
                    mark = 1
                    new_lst.append(1)
                    break
                elif((max_value == 0 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 0 and max_value == 4)):
                    mark = 1
                    new_lst.append(5)
                    break
                elif((max_value == 1 and lst[i].index(lst[i][k]) == 2) or (lst[i].index(lst[i][k]) == 1 and max_value == 2)):
                    mark = 1
                    new_lst.append(3)
                    break
                elif((max_value == 1 and lst[i].index(lst[i][k]) == 3) or (lst[i].index(lst[i][k]) == 1 and max_value == 3)):
                    mark = 1
                    new_lst.append(4)
                    break
                elif((max_value == 1 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 1 and max_value == 4)):
                    mark = 1
                    new_lst.append(5)
                    break
                elif((max_value == 2 and lst[i].index(lst[i][k]) == 3) or (lst[i].index(lst[i][k]) == 2 and max_value == 3)):
                    mark = 1
                    new_lst.append(3)
                    break
                elif((max_value == 2 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 2 and max_value == 4)):
                    mark = 1
                    new_lst.append(5)
                    break
                elif((max_value == 3 and lst[i].index(lst[i][k]) == 4) or (lst[i].index(lst[i][k]) == 3 and max_value == 4)):
                    mark = 1
                    new_lst.append(5)
                    break
    
    if(mark == 0):
        max_value = max_value + 1
        new_lst.append(max_value)
        
for i in range(len(new_lst)):
        print(new_lst[i])
        
print(len(new_lst))

2256
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
3
3
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
3
5
5
5
5
5
5
5
3
1
1
3
3
3
4
1
1
4
1
1
4
3
2
3
3
3
3
3
3
3
1
1
5
5
5
5
5
5
5
5
5
2
5
1
2
2
2
2
3
5
1
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
1
1
5
1
5
5
5
5
5
5
5
5
3
3
3
5
5
5
2
5
1
2
2
2
3
3
1
1
1
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
3
3
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
3
5
5
5
5
5
5
5
3
1
1
3
3
3
4
1
1
4
1
1
4
3
2
3
3
3
3
3
3
3
1
1
5
5
5
5
5
5
5
5
5
2
5
1
2
2
2
2
3
5
1
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
1
1
5
1
5
5
5
5
5
5
5
5
3
3
3
5
5
5
2
5
1
2
2
2
3
3
1
1
1
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
3
3
5
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
2
5
3
5
5
5
5
5
5
5
3
1
1
3
3
3
4
1
1
4
1
1
4
3
2
3
3
3
3
3
3
3
1
1
5
5
5
5
5
5
5
5
5
2
5
1
2
2
2
2
3
5
1
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
1
1
1
5
1
5
5
5
5
5
5
5
5
3
3
3
5
5
5
2
5
1
2
2
2
3
3
1